In [ ]:
# imports

import os
from pathlib import Path
from omegaconf import OmegaConf

import torch

from src.data_pipeline import CIFAR10Pipeline
from src.classifier import Classifier
from src.runner import run_experiment, compare_experiments

In [ ]:
# load config

cfg = OmegaConf.load("conf/config.yaml")

print("="*60)
print("Setting up data pipeline...")
print("="*60)

data_pipeline = CIFAR10Pipeline(cfg)
train_loader, val_loader, test_loader = data_pipeline.setup()

print(f"\nDataset: CIFAR-10")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")
print(f"Batch size: {cfg.train.batch_size}\n")

In [ ]:
# initialize experiment params

# Dictionary to store results from all experiments
experiment_results = {}

# Run experiment for the configured model
model_name = cfg.model.model_name
print("="*60)
print(f"Running experiment: {model_name}")
print("="*60)

In [ ]:
# run single experiment

cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

results = run_experiment(
    cfg=cfg,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=cifar10_classes,
    pretrained=False,
    freeze_backbone=False
)

experiment_results[model_name] = results

print("\n" + "="*60)
print("Experiment completed successfully!")
print("="*60)

In [ ]:
# run multiple experminets

experiments = [
    # Custom CNN from scratch
    {'model_name': 'CustomCNN', 'pretrained': False, 'freeze_backbone': False},

    # Pretrained ResNet18 with fine-tuning (all layers trainable)
    {'model_name': 'ResNet18', 'pretrained': True, 'freeze_backbone': False},

    # Pretrained ResNet18 frozen (only classifier head trainable)
    {'model_name': 'ResNet18', 'pretrained': True, 'freeze_backbone': True},
]

for exp in experiments:
    cfg.model.model_name = exp['model_name']
    print("\n" + "="*60)
    print(f"Running experiment: {exp['model_name']} (pretrained={exp['pretrained']}, frozen={exp['freeze_backbone']})")
    print("="*60)

    results = run_experiment(
        cfg=cfg,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        class_names=cifar10_classes,
        pretrained=exp['pretrained'],
        freeze_backbone=exp['freeze_backbone']
    )

    exp_name = results['model_name']
    experiment_results[exp_name] = results

# Compare all experiments
compare_experiments(experiment_results, save_path="results/model_comparison.png")